## Initializing Sources and "Interesting Words"

In [20]:
import mediacloud
import os
import json
import csv
import requests
import collections


In [21]:
MY_API_KEY = '0b048304d2f7398cb91248b7e07b3b153d32840c1a8c42ab4006f58aaa8a440a'

In [22]:
left_media = ["New York Times", "NPR", "Politico", "CNN", "American Civil Liberties Union"]
left_fake_media = ["If You Only News", "Occupy Democrats", "FWD Now", "Jezebel", "Blue Tribune"]
right_media = ["Fox News", "Daily Telegraph", "Chicago Tribune", "Forbes", "Washington Times"]
right_fake_media = ["Russia Today", "InfoWars", "Natural News", "DCLeaks", "Breitbart"]

In [23]:
random_interesting_words = ['Trump', 'Obama', 'fake news', 'terrorist', 'Kavanaugh', 'midterm', 'Republican', 'Democrat', 'election',
                     'Russia', 'Jeff Sessions', 'Attorney General', 'tolerance', 'racism', 'sexism', 'gender', 'snowflake',
                     'shooting', 'massacre', 'guns', 'abortion', 'radical', 'leftwing', 'rightwing', 'queer', 'gay', 'religion',
                     'healthcare', 'universal', 'immigrant', 'refugee', 'Syria', 'education']

## Mapping Media Names to Mediacloud ID's

In [24]:
def write_csv_media(media_name, rows=100):
    """
    Args: media_names: media source name (can have multiple) in list format
          chunk_size: max # of media sources that can be written per iteration
    Write the media_id, url, and names of the given media_name(s) in a CSV file
    """
    media = []
    media_idx = 0
    last_media_id = 0

    fieldnames = [
        u'media_id',
        u'url',
        u'name'
    ]

    while True:
        params = { 'last_media_id': last_media_id, 'rows': rows, 'name': media_name[media_idx], 'key': MY_API_KEY }
        media_list_call = 'https://api.mediacloud.org/api/v2/media/list'
        r = requests.get( media_list_call, params = params, headers = { 'Accept': 'application/json'} )
        data = r.json()
        print ("start:{} num_media_sources:{}".format(last_media_id, len(data)))

        if not len(data):

            # If there are more media_names, write what we have to csv file and continue
            path_name = './csv_storage/media.csv'
            with open( path_name, 'a', newline="") as csvfile:
                print ("\nOpened file: Dumping media source content for {}\n".format(media_name[media_idx]))

                # Flush media buffer to csv file
                cwriter = csv.DictWriter( csvfile, fieldnames, extrasaction='ignore')

                if not os.path.getsize(path_name):
                    cwriter.writeheader()

                cwriter.writerows( media )

            # Continue to next user-inputted media_name
            media_idx += 1
            last_media_id = 0
            media = []
            if media_idx < len(media_name):
                print ("Grabbing sources of next media name:{}\n".format(media_name[media_idx]))
                continue

            # Done if no more media sources to get
            break


        #add to media buffer and search for more media sources similar to current media_name
        media.extend( data )
        last_media_id = media[-1]['media_id']
 

In [12]:
write_csv_media(left_media)
write_csv_media(left_fake_media)
write_csv_media(right_media)
write_csv_media(right_fake_media)

start:0 rows:100
start:651204 rows:100

Opened file: Dumping media source content for New York Times

Grabbing sources of next media name:NPR

start:0 rows:100
start:91179 rows:100
start:139914 rows:100
start:190302 rows:100
start:240503 rows:100
start:299186 rows:100
start:369062 rows:100
start:443490 rows:100
start:508448 rows:100
start:558433 rows:100
start:616090 rows:100
start:705557 rows:100
start:763846 rows:100
start:821041 rows:100
start:885800 rows:100
start:944458 rows:100
start:1011207 rows:100
start:1013589 rows:100

Opened file: Dumping media source content for NPR

Grabbing sources of next media name:Politico

start:0 rows:100
start:345823 rows:100
start:958555 rows:100
start:990826 rows:100

Opened file: Dumping media source content for Politico

Grabbing sources of next media name:CNN

start:0 rows:100
start:268057 rows:100
start:780894 rows:100
start:999468 rows:100

Opened file: Dumping media source content for CNN

Grabbing sources of next media name:American Civil 

In [6]:
#Media_ID's with respect to their names in the above lists. Fill in later.
left_media_dict = {}
left_fake_media_dict = {}
right_media_dict = {}
right_fake_media_dict = {}


## Word Counting and Sentiment Analysis

Define functions here to loop through the media_sources, grab word-count information, and the assign a positive/negative sentiment score. 

In [39]:
def get_story_word_matrix(media_id, rows=1000):
    """
    Args: media_id: ID of media source to grab word matrices for. Stories in the returned dicts
    will only be from this media_id.
    """
    params = {'rows': rows, 'q': 'media_id: {}'.format(media_id), 'key': MY_API_KEY}
    word_matrix_call = 'https://api.mediacloud.org/api/v2/stories_public/word_matrix'
    r = requests.get(word_matrix_call, params = params, headers = { 'Accept': 'application/json'} )
    data = r.json()
    return data['word_list'], data['word_matrix']

In [46]:
def get_story_word_matrices_for_all_media(media_dict):
    word_lists_and_matrices = []
    for media_id in media_dict.keys():
        data = get_story_word_matrix(media_id)
        print ('Word list for media source: {}\n\n'.format(media_dict[media_id]))
        print (data['word_list'])
        print ('Word matrix of word frequency counts by stories_ids for {}\n\n'.format(media_dict[media_id]))
        print (data['word_matrix'])
        word_lists_and_matrices.append(data)
    return word_lists_and_matrices

SyntaxError: invalid syntax (<ipython-input-46-a9fc7c393e52>, line 5)

In [47]:
left_media_data = get_story_word_matrices_for_all_media(left_media_dict)
left_fake_media_data = get_story_word_matrices_for_all_media(left_fake_media_dict)
right_media_data = get_story_word_matrices_for_all_media(right_media_dict)
right_fake_media_data = get_story_word_matrices_for_all_media(right_fake_media_dict)

NameError: name 'get_story_word_matrices_for_all_media' is not defined

In [48]:
def get_repeat_words_within_media(data, threshold=10):
    """
    Returns a dictionary with repeat words as a key mapped to the number of stories 
    they've appeared in as a repeat word.
    """
    stories_ids = data['word_matrix'].keys()
    repeat_words = collections.Counter()
    for stories_id in stories_ids:
        word_ids = data['word_matrix'][stories_id].keys()
        for word_id in word_ids:
            if data['word_matrix'][stories_id][word_id] >= threshold:
                repeat_words[data['word_list'][int(word_id)]] += 1
    return repeat_words
        

In [8]:
def get_repeat_words_within_bias(bias_data):
    repeat_words_across_bias = []
    for data in bias_data: 
        repeat_words = get_repeat_words_within_media(data)
        repeat_words_across_bias.append(repeat_words)
        
    repeat_words_
    for repeat_words in repeat_words_across_bias[1:]:
        repeat_words_set = set(repeat_words.keys())
        if 
        
                
    
    stem = 

ValueError: Expecting value: line 1 column 1 (char 0)

In [ ]:
def get_word_count_for_media(media_id, word, num_words=500, sample_size=100000): 
    """
    Args: media_id: ID of media_source to grab word count of interesting words for. 
    """
    params = {'num_words': num_words, 'sample_size': sample_size, 'include_stats': 1, 
              'q': 'media_id: {} AND {}'.format(media_id, word)}
    word_count_call = 'https://api.mediacloud.org/api/v2/wc/list'
    r = requests.get(word_count_call, params = params, headers = { 'Accept': 'application/json'} )
    data = r.json()
    return data

In [ ]:
def get_all_interesting_word_counts_for_all_media(media_dict):
    for media_id in media_dict.keys():
        for word in interesting_words:
            data = get_word_count_for_media(media_id, word)
            print "Word frequency counts in sentences containing the word {} in {}:\n\n".format(word, media_dict[media_id])
            print data
    

In [ ]:
get_all_interesting_word_counts_for_all_media(media_dict)